In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkinter.simpledialog import askstring
from tkinter.filedialog import asksaveasfilename
from tkinter.filedialog import askopenfilename
from tkcalendar import Calendar
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import datetime
import csv
from collections import defaultdict
import hashlib
import numpy as np

# ---------------- Data Storage ----------------
expenses = []
recurring_categories = set()

# ---------------- App Core ----------------
class ExpenseTrackerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("TrackWise - Expense Tracker")
        self.geometry("950x600")
        self.config(bg="#1f1f2e")
        self.iconbitmap(r"C:\Users\Eol\Downloads\istockphoto-1353574001-170667a.ico")

        self.frames = {}
        for F in (HomePage, AddExpensePage, ViewExpensesPage, PieChartPage, FilterPage,
                  ExportPage, MonthlyChartPage, DateRangeFilterPage, RecurringAlertPage, CalendarViewPage,ImportPage,UpdateExpensePage,DeleteExpensePage):
            frame = F(parent=self, controller=self)
            self.frames[F] = frame
            frame.place(x=160, y=0, relwidth=1, relheight=1)

        self.show_frame(HomePage)
        self.create_nav_panel()

    def show_frame(self, page):
        frame = self.frames[page]
        frame.tkraise()
        frame.event_generate("<<ShowFrame>>")

    def create_nav_panel(self):
        nav = tk.Frame(self, bg="#141421", width=160)
        nav.pack(side="left", fill="y")

        buttons = [
            ("🏠 Home", HomePage),
            ("➕ Add", AddExpensePage),
            ("✏️ Update Amount", UpdateExpensePage),
            ("🔍 Filter", FilterPage),
            ("📋 View", ViewExpensesPage),
            ("📅 Calendar", CalendarViewPage),
            ("📊 Pie", PieChartPage),
            ("📅 Monthly", MonthlyChartPage),
            ("📆 Date Range", DateRangeFilterPage),
            ("🔔 Alerts", RecurringAlertPage),
            ("📤 Export", ExportPage),
            ("📁 Import CSV", ImportPage),
            ("❌ Delete", DeleteExpensePage)
        ]

        for text, page in buttons:
            tk.Button(nav, text=text, command=lambda p=page: self.show_frame(p),
                      bg="#333", fg="white", font=("Arial", 10), relief="flat",
                      activebackground="#555").pack(fill='x', pady=2, padx=5)

# ---------------- Shared TreeView Setup ----------------
def setup_treeview(frame, columns):
    tree = ttk.Treeview(frame, columns=columns + [""], show='headings')
    for col in columns:
        tree.heading(col, text=col)
        width = 140 if col != "Note" else 320
        tree.column(col, anchor="center", width=width)
    tree.column("", anchor="center", width=2, stretch=False)
    tree.heading("", text="")
    tree.pack(expand=True, fill="both")
    style = ttk.Style()
    style.theme_use("clam")
    style.configure("Treeview", background="#2a2a3d", foreground="white",
                rowheight=28, fieldbackground="#2a2a3d", font=("Segoe UI", 10))
    style.configure("Treeview.Heading", font=('Segoe UI', 11, 'bold'),
                background="#1a1a2e", foreground="#ffd369")

    return tree

# ---------------- Pages ----------------

class HomePage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        tk.Label(self, text="Welcome to TrackWise Expense Tracker", font=("Arial", 18, 'bold'),
                 fg="white", bg="#2e2e2e").pack(pady=50)
        tk.Label(self, text="Use the navigation menu on the left.", font=("Arial", 12),
                 fg="white", bg="#2e2e2e").pack()

class AddExpensePage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")

        def add():
            try:
                date = date_entry.get()
                datetime.datetime.strptime(date, '%Y-%m-%d')
                category = category_entry.get()
                amount = float(amount_entry.get())
                note = note_entry.get()
                expenses.append({"date": date, "category": category, "amount": amount, "note": note})
                messagebox.showinfo("Success", "Expense added!")
                clear()
            except:
                messagebox.showerror("Error", "Invalid input!")

        def clear():
            for entry in [date_entry, category_entry, amount_entry, note_entry]:
                entry.delete(0, tk.END)

        tk.Label(self, text="Add Expense", font=("Arial", 16, "bold"), fg="white", bg="#2e2e2e").pack(pady=10)
        form = tk.Frame(self, bg="#2e2e2e", highlightbackground="white", highlightthickness=1)
        form.pack(pady=20, padx=50)

        def row(label, r):
            tk.Label(form, text=label, fg="white", bg="#2e2e2e", font=("Arial", 10)).grid(row=r, column=0, padx=10, pady=5, sticky='e')
            entry = tk.Entry(form, width=30, bg="#444", fg="white", insertbackground="white")
            entry.grid(row=r, column=1, padx=10, pady=5)
            return entry

        date_entry = row("Date (YYYY-MM-DD):", 0)
        category_entry = row("Category:", 1)
        amount_entry = row("Amount:", 2)
        note_entry = row("Note:", 3)

        tk.Button(self, text="Add", command=add, bg="#555", fg="white", font=("Arial", 10, "bold"), width=20)\
            .pack(pady=10)

class ViewExpensesPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        tk.Label(self, text="All Expenses", font=("Arial", 16, "bold"), fg="white", bg="#2e2e2e").pack(pady=10)

        frame = tk.Frame(self, bg="#2e2e2e")
        frame.pack(expand=True, fill="both", padx=50, pady=10)

        self.tree = setup_treeview(frame, ["Date", "Category", "Amount", "Note"])

        # Show Averages Button
        tk.Button(self, text="📈 Show Averages", command=self.show_category_averages,
                  bg="#555", fg="white", font=("Arial", 10)).pack(pady=5)

        self.bind("<<ShowFrame>>", self.refresh)

    def refresh(self, event=None):
        self.tree.delete(*self.tree.get_children())
        for e in expenses:
            self.tree.insert("", "end", values=(e["date"], e["category"], f"{e['amount']:.2f}", e["note"], ""))

    def show_category_averages(self):
        data = defaultdict(list)
        for e in expenses:
            data[e["category"]].append(e["amount"])

        if not data:
            messagebox.showinfo("No Data", "No expenses to calculate averages.")
            return

        import numpy as np
        result = "\n".join(f"{cat}: {np.mean(values):.2f}" for cat, values in data.items())
        messagebox.showinfo("Category Averages", result)


class PieChartPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        tk.Label(self, text="Pie Chart", font=("Arial", 16, "bold"), fg="white", bg="#2e2e2e").pack(pady=10)
        self.wrapper = tk.Frame(self, bg="#2e2e2e", highlightbackground="white", highlightthickness=1)
        self.wrapper.pack(padx=50, pady=20)
        self.canvas = None

        tk.Button(self, text="Generate", command=self.draw_chart,
                  bg="#555", fg="white", font=("Arial", 10, "bold")).pack(pady=10)

        self.bind("<<ShowFrame>>", lambda e: self.clear())

    def clear(self):
        if self.canvas:
            self.canvas.get_tk_widget().destroy()
            self.canvas = None

    def draw_chart(self):
        self.clear()
        if not expenses:
            messagebox.showinfo("Info", "No data")
            return
        data = defaultdict(float)
        for e in expenses:
            data[e["category"]] += e["amount"]
        fig, ax = plt.subplots()
        ax.pie(data.values(), labels=data.keys(), autopct='%1.1f%%')
        ax.set_title("Expenses by Category")
        self.canvas = FigureCanvasTkAgg(fig, master=self.wrapper)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack()

class FilterPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        tk.Label(self, text="Filter by Category", font=("Arial", 16, "bold"), fg="white", bg="#2e2e2e").pack(pady=10)
        frame = tk.Frame(self, bg="#2e2e2e")
        frame.pack(expand=True, fill="both", padx=50, pady=10)
        self.tree = setup_treeview(frame, ["Date", "Category", "Amount", "Note"])

        tk.Button(self, text="Enter Category", command=self.filter,
                  bg="#555", fg="white", font=("Arial", 10, "bold")).pack(pady=10)

        self.bind("<<ShowFrame>>", self.clear)

    def clear(self, e=None):
        self.tree.delete(*self.tree.get_children())

    def filter(self):
        category = askstring("Category", "Enter category:")
        if not category:
            return
        filtered = [e for e in expenses if e["category"].lower() == category.lower()]
        self.clear()
        for e in filtered:
            self.tree.insert("", "end", values=(e["date"], e["category"], f"{e['amount']:.2f}", e["note"], ""))

class ExportPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        tk.Label(self, text="Export Expenses", font=("Arial", 16, "bold"), fg="white", bg="#2e2e2e").pack(pady=20)
        tk.Button(self, text="Export to CSV", command=self.export_csv,
                  bg="#555", fg="white", font=("Arial", 10, "bold"), width=25).pack(pady=10)

    def export_csv(self):
        path = asksaveasfilename(defaultextension=".csv", filetypes=[("CSV Files", "*.csv")])
        if not path:
            return
        with open(path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Date", "Category", "Amount", "Note"])
            for e in expenses:
                writer.writerow([e["date"], e["category"], e["amount"], e["note"]])
        messagebox.showinfo("Done", "Expenses exported successfully.")

class MonthlyChartPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        tk.Label(self, text="Monthly Expense Chart", font=("Arial", 16, "bold"), fg="white", bg="#2e2e2e").pack(pady=10)

        self.wrapper = tk.Frame(self, bg="#2e2e2e", highlightbackground="white", highlightthickness=1)
        self.wrapper.pack(padx=50, pady=20)

        self.canvas = None

        tk.Button(self, text="Generate", command=self.draw_chart,
                  bg="#555", fg="white", font=("Arial", 10, "bold")).pack(pady=10)

        self.bind("<<ShowFrame>>", lambda e: self.clear())

    def clear(self):
        if self.canvas:
            self.canvas.get_tk_widget().destroy()
            self.canvas = None

    def draw_chart(self):
        self.clear()
        monthly = defaultdict(float)
        for e in expenses:
            month = e["date"][:7]  # Extract YYYY-MM from date
            monthly[month] += e["amount"]

        if not monthly:
            messagebox.showinfo("No Data", "No expenses available.")
            return

        months = list(monthly.keys())
        values = list(monthly.values())
        mean_value = np.mean(values)

        fig, ax = plt.subplots()
        ax.bar(months, values)
        ax.axhline(mean_value, color='red', linestyle='--', label=f'Mean: {mean_value:.2f}')
        ax.set_title("Total per Month")
        ax.legend()

        self.canvas = FigureCanvasTkAgg(fig, master=self.wrapper)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack()

class DateRangeFilterPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        tk.Label(self, text="Filter by Date Range", font=("Arial", 16, "bold"), fg="white", bg="#2e2e2e").pack(pady=10)

        frame = tk.Frame(self, bg="#2e2e2e")
        frame.pack(expand=True, fill="both", padx=50, pady=10)
        self.tree = setup_treeview(frame, ["Date", "Category", "Amount", "Note"])

        tk.Button(self, text="Filter", command=self.filter_range,
                  bg="#555", fg="white", font=("Arial", 10, "bold")).pack(pady=10)

        self.bind("<<ShowFrame>>", lambda e: self.tree.delete(*self.tree.get_children()))

    def filter_range(self):
        start = askstring("Start Date", "Enter start date (YYYY-MM-DD):")
        end = askstring("End Date", "Enter end date (YYYY-MM-DD):")

        # Validate inputs
        try:
            start_date = datetime.datetime.strptime(start.strip(), "%Y-%m-%d")
            end_date = datetime.datetime.strptime(end.strip(), "%Y-%m-%d")
        except:
            messagebox.showerror("Date Error", "Please enter valid dates in YYYY-MM-DD format.")
            return

        if start_date > end_date:
            messagebox.showerror("Range Error", "Start date must be before end date.")
            return

        self.tree.delete(*self.tree.get_children())
        found = False
        for e in expenses:
            try:
                e_date = datetime.datetime.strptime(e["date"], "%Y-%m-%d")
                if start_date <= e_date <= end_date:
                    self.tree.insert("", "end", values=(e["date"], e["category"], f"{e['amount']:.2f}", e["note"], ""))
                    found = True
            except:
                continue

        if not found:
            messagebox.showinfo("No Results", "No expenses found in this date range.")


class RecurringAlertPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        tk.Label(self, text="Recurring Expense Alerts", font=("Arial", 16, "bold"), fg="white", bg="#2e2e2e").pack(pady=10)
        tk.Button(self, text="Set Recurring Category", command=self.set_recurring,
                  bg="#555", fg="white", font=("Arial", 10, "bold")).pack(pady=10)
        self.label = tk.Label(self, text="", fg="red", bg="#2e2e2e", font=("Arial", 12))
        self.label.pack()
        self.bind("<<ShowFrame>>", self.check_alerts)

    def set_recurring(self):
        category = askstring("Category", "Enter recurring category:")
        if category:
            recurring_categories.add(category.lower())
            messagebox.showinfo("Added", f"{category} marked as recurring.")

    def check_alerts(self, event=None):
        found = {e["category"].lower() for e in expenses}
        alerts = [cat for cat in recurring_categories if cat not in found]
        self.label.config(text="\n".join(f"⚠️ Missing: {cat.title()}" for cat in alerts) if alerts else "✅ All recurring categories are recorded.")

class ImportPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        self.controller = controller

        tk.Label(self, text="📁 Import CSV File", font=("Arial", 16, "bold"),
                 fg="white", bg="#2e2e2e").pack(pady=20)

        tk.Button(self, text="Select CSV File", command=self.import_csv,
                  bg="#555", fg="white", font=("Arial", 10, "bold"), width=25).pack(pady=10)

    def import_csv(self):
        path = askopenfilename(filetypes=[("CSV Files", "*.csv")])
        if not path:
            return

        imported = 0
        with open(path, newline='') as file:
            reader = csv.DictReader(file)
            for row in reader:
                try:
                    date = row["Date"].strip()
                    datetime.datetime.strptime(date, "%Y-%m-%d")  # validate
                    category = row["Category"].strip()
                    amount = float(row["Amount"].strip())
                    note = row["Note"].strip()
                    expenses.append({"date": date, "category": category, "amount": amount, "note": note})
                    imported += 1
                except Exception as e:
                    print("Skipping row due to error:", e)
                    continue

        messagebox.showinfo("Imported", f"{imported} expenses added successfully.")

        # Trigger refresh for key pages
        self.controller.frames[PieChartPage].clear()
        self.controller.frames[MonthlyChartPage].clear()
        self.controller.frames[ViewExpensesPage].refresh()
        self.controller.frames[CalendarViewPage].show_expenses()

class CalendarViewPage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")

        tk.Label(self, text="📅 Calendar View", font=("Arial", 16, "bold"),
                 fg="white", bg="#2e2e2e").pack(pady=10)

        # Use correct date pattern (yyyy-mm-dd) to match CSV and internal data
        self.calendar = Calendar(self, selectmode='day', date_pattern='yyyy-mm-dd')
        self.calendar.pack(pady=20)

        tk.Button(self, text="View Expenses for Date", command=self.show_expenses,
                  bg="#555", fg="white", font=("Arial", 10, "bold")).pack(pady=5)

        tk.Button(self, text="Add Expense for Date", command=self.add_expense,
                  bg="#555", fg="white", font=("Arial", 10)).pack(pady=5)

        # Table for selected date
        frame = tk.Frame(self, bg="#2e2e2e")
        frame.pack(expand=True, fill="both", padx=50, pady=10)
        self.tree = setup_treeview(frame, ["Date", "Category", "Amount", "Note"])

        self.bind("<<ShowFrame>>", lambda e: self.tree.delete(*self.tree.get_children()))

    def show_expenses(self):
        selected_date = self.calendar.get_date()  # Already yyyy-mm-dd
        self.tree.delete(*self.tree.get_children())
        for e in expenses:
            # Normalize CSV date like 2025-1-1 to 2025-01-01
            try:
                normalized = datetime.datetime.strptime(e["date"], "%Y-%m-%d").strftime("%Y-%m-%d")
            except:
                continue
            if normalized == selected_date:
                self.tree.insert("", "end", values=(e["date"], e["category"], f"{e['amount']:.2f}", e["note"], ""))


    def add_expense(self):
        selected_date = self.calendar.get_date()  # Already in yyyy-mm-dd format
        category = askstring("Category", f"Enter category for {selected_date}:")
        if not category:
            return
        try:
            amount_str = askstring("Amount", f"Enter amount for {selected_date}:")
            amount = float(amount_str)
        except:
            messagebox.showerror("Error", "Invalid amount.")
            return
        note = askstring("Note", "Enter note (optional):") or ""
        expenses.append({"date": selected_date, "category": category, "amount": amount, "note": note})
        messagebox.showinfo("Added", f"Expense for {selected_date} added.")
        self.show_expenses()

class UpdateExpensePage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        self.controller = controller

        tk.Label(self, text="🔄 Update Expense Amount", font=("Arial", 16, "bold"),
                 fg="white", bg="#2e2e2e").pack(pady=10)

        self.category_entry = tk.Entry(self, width=30, bg="#444", fg="white", font=("Arial", 11))
        self.category_entry.pack(pady=5)
        self.category_entry.insert(0, "Enter Category to Search")

        tk.Button(self, text="Search", command=self.search_expenses,
                  bg="#555", fg="white", font=("Arial", 10)).pack(pady=5)

        # Treeview to display results
        self.tree_frame = tk.Frame(self, bg="#2e2e2e")
        self.tree_frame.pack(expand=True, fill="both", padx=50, pady=10)
        self.tree = setup_treeview(self.tree_frame, ["Date", "Category", "Amount", "Note"])

        tk.Button(self, text="Update Selected", command=self.update_selected,
                  bg="#555", fg="white", font=("Arial", 10)).pack(pady=10)

    def search_expenses(self):
        category = self.category_entry.get().strip().lower()
        self.tree.delete(*self.tree.get_children())
        for i, e in enumerate(expenses):
            if e["category"].lower() == category:
                self.tree.insert("", "end", iid=str(i), values=(e["date"], e["category"], f"{e['amount']:.2f}", e["note"], ""))

    def update_selected(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showerror("Select Entry", "Please select an expense to update.")
            return

        index = int(selected[0])
        new_amount = askstring("New Amount", f"Enter new amount for {expenses[index]['category']}:")

        try:
            expenses[index]["amount"] = float(new_amount)
            self.search_expenses()
            messagebox.showinfo("Updated", "Amount updated successfully.")
        except:
            messagebox.showerror("Invalid Input", "Please enter a valid number.")

class DeleteExpensePage(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent, bg="#2e2e2e")
        self.controller = controller

        tk.Label(self, text="🗑️ Delete Expense", font=("Arial", 16, "bold"),
                 fg="white", bg="#2e2e2e").pack(pady=10)

        self.category_entry = tk.Entry(self, width=30, bg="#444", fg="white", font=("Arial", 11))
        self.category_entry.pack(pady=5)
        self.category_entry.insert(0, "Enter Category to Search")

        tk.Button(self, text="Search", command=self.search_expenses,
                  bg="#555", fg="white", font=("Arial", 10)).pack(pady=5)

        self.tree_frame = tk.Frame(self, bg="#2e2e2e")
        self.tree_frame.pack(expand=True, fill="both", padx=50, pady=10)
        self.tree = setup_treeview(self.tree_frame, ["Date", "Category", "Amount", "Note"])

        tk.Button(self, text="Delete Selected", command=self.delete_selected,
                  bg="#aa4444", fg="white", font=("Arial", 10)).pack(pady=10)

    def search_expenses(self):
        category = self.category_entry.get().strip().lower()
        self.tree.delete(*self.tree.get_children())
        for i, e in enumerate(expenses):
            if e["category"].lower() == category:
                self.tree.insert("", "end", iid=str(i), values=(e["date"], e["category"], f"{e['amount']:.2f}", e["note"], ""))

    def delete_selected(self):
        selected = self.tree.selection()
        if not selected:
            messagebox.showerror("Select Entry", "Please select an expense to delete.")
            return

        index = int(selected[0])
        confirm = messagebox.askyesno("Confirm Delete", f"Are you sure you want to delete this expense?\n\n{expenses[index]}")
        if confirm:
            del expenses[index]
            self.search_expenses()
            messagebox.showinfo("Deleted", "Expense deleted successfully.")

# ---------------- Launch App ----------------
def verify_password(entered_password):
    stored_hash = "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8"  # hash for "password"
    entered_hash = hashlib.sha256(entered_password.encode()).hexdigest()
    return entered_hash == stored_hash

def launch_login():
    login = tk.Tk()
    login.title("TrackWise Login")
    login.geometry("300x180")
    login.config(bg="#2e2e2e")
    login.iconbitmap(r"C:\Users\Eol\Downloads\istockphoto-1353574001-170667a.ico")

    tk.Label(login, text="Enter Password", bg="#2e2e2e", fg="white", font=("Arial", 12)).pack(pady=20)
    pwd_entry = tk.Entry(login, show="*", width=25, bg="#444", fg="white", font=("Arial", 11), insertbackground="white")
    pwd_entry.pack(pady=5)

    def attempt_login():
        if verify_password(pwd_entry.get()):
            login.destroy()
            app = ExpenseTrackerApp()
            app.mainloop()
        else:
            messagebox.showerror("Access Denied", "Incorrect password.")

    tk.Button(login, text="Login", command=attempt_login,
              bg="#555", fg="white", font=("Arial", 10, "bold")).pack(pady=20)

    login.mainloop()

if __name__ == "__main__":
    launch_login()